In [31]:
import torch
import boto3
session = boto3.Session(profile_name="a1")

In [29]:
dynamodb = session.client('dynamodb', region_name="us-east-1")
table_name = "oslynstudio-RecordingTable"
items = dynamodb.scan(
    TableName=table_name,
    Select="ALL_ATTRIBUTES"
)


1.36.16
DynamoDB Tables: {'TableNames': ['oslynstudio-BandTable', 'oslynstudio-BandUserRoleTable', 'oslynstudio-InviteTable', 'oslynstudio-JamSessionTable', 'oslynstudio-RecordingTable', 'oslynstudio-SetListTable', 'oslynstudio-SongTable', 'oslynstudio-UserTable'], 'ResponseMetadata': {'RequestId': 'O6DDR3DVLBGN431SN2I9KCHTERVV4KQNSO5AEMVJF66Q9ASUAAJG', 'HTTPStatusCode': 200, 'HTTPHeaders': {'server': 'Server', 'date': 'Sat, 08 Feb 2025 21:52:58 GMT', 'content-type': 'application/x-amz-json-1.0', 'content-length': '232', 'connection': 'keep-alive', 'x-amzn-requestid': 'O6DDR3DVLBGN431SN2I9KCHTERVV4KQNSO5AEMVJF66Q9ASUAAJG', 'x-amz-crc32': '1916392939'}, 'RetryAttempts': 0}}


In [42]:
from boto3.dynamodb.types import TypeDeserializer
unmarshall = TypeDeserializer()

## Migration, coppy all the data fro old table to backup table
dynamodb = session.client('dynamodb', region_name="us-east-1")

# Old and new table references
old_table = "oslynstudio-RecordingTable"
new_table = "RecordingTableBackupFeb2025"

# Scan old table to get all items
items = dynamodb.scan(
    TableName=old_table,
    Select="ALL_ATTRIBUTES"
).get("Items")

print(items)

# Migrate each item to the new table
for item in items:
    # Ensure updateDate is set (use existing timestamp or generate new one)
    if "updateDate" not in item:
        raise Exception("There should not be updateDate that are empty!")
        #item["updateDate"] = int(time.time())  # Assign current timestamp if missing
    
    response = dynamodb.put_item(TableName=new_table, Item=item)
    print(response)

print(f"Migrated {len(items)} items successfully!")



[{'songs': {'L': [{'M': {'pageturns': {'L': []}, 'songId': {'S': '17eb5cb3-790a-4021-aa5a-4f6ecd66e5a8'}, 'startTime': {'S': '0'}}}]}, 'updateDate': {'N': '1724027244374'}, 'fileName': {'S': 'recordings/50dc3ed6-5cfd-4f26-b906-687c16f0fe66/2acfed71-cc67-4b39-a8a4-83ae4c0511e2/002_985c57e6-6312-4959-91b5-ea4c68091113_2024-08-19T00:27:24.081Z.wav'}, 'createDate': {'N': '1724027244374'}, 'sessionId': {'S': '2acfed71-cc67-4b39-a8a4-83ae4c0511e2'}, 'userId': {'S': '9e7724c6-fbda-4a0c-874d-3841ae0848c1_usr'}, 'status': {'S': ''}, 'jamId': {'S': '50dc3ed6-5cfd-4f26-b906-687c16f0fe66'}, 'comment': {'S': ''}, 'recordingId': {'S': '985c57e6-6312-4959-91b5-ea4c68091113'}, 'samplingRate': {'N': '16000'}}, {'songs': {'L': [{'M': {'pageturns': {'L': []}, 'songId': {'S': '8e3eba70-458f-4a1d-a2a4-7abd9e5d1495'}, 'startTime': {'S': '0'}}}]}, 'updateDate': {'N': '1724016204835'}, 'fileName': {'S': 'recordings/50dc3ed6-5cfd-4f26-b906-687c16f0fe66/7a0c584f-eefb-4e53-8173-caad070a492b/002_ea4dadf7-b5ea-42b